In [1]:
import org.apache.spark.sql.SparkSession;

//import statistics.functions._;

import org.apache.spark.sql.DataFrame;
import org.apache.spark.sql.types._;
import org.apache.spark.sql.functions._;
import org.apache.spark.ml.Pipeline;
import org.apache.spark.ml.Model;
import org.apache.spark.ml.classification.LogisticRegression;
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator;
import org.apache.spark.ml.feature.Binarizer;
import org.apache.spark.ml.feature.{RegexTokenizer, NGram};
import org.apache.spark.ml.feature.{HashingTF, IDF};
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder};

Intitializing Scala interpreter ...

Spark Web UI available at http://pranav-pc:4041
SparkContext available as 'sc' (version = 3.0.0-preview2, master = local[*], app id = local-1580823277376)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.Model
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.Binarizer
import org.apache.spark.ml.feature.{RegexTokenizer, NGram}
import org.apache.spark.ml.feature.{HashingTF, IDF}
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}


In [2]:
val path: String = "hdfs://localhost:9000/TextMining/tokens/part-00000";

// "rate" must be Double because it can be easily binarized by Spark
val original_schema = new StructType(Array(
  StructField("product", StringType,  true),
  StructField("votes",   IntegerType, true),
  StructField("rate",    DoubleType,  true),
  StructField("original_text",    StringType,  true),
  StructField("text",    StringType,  true),
  StructField("summary",    StringType,  true)));

val original_data:DataFrame = spark.read
  .options(Map("delimiter" -> "\t"))
  .schema(original_schema)
  .csv(path)
  .na.drop();

path: String = hdfs://localhost:9000/TextMining/tokens/part-00000
original_schema: org.apache.spark.sql.types.StructType = StructType(StructField(product,StringType,true), StructField(votes,IntegerType,true), StructField(rate,DoubleType,true), StructField(original_text,StringType,true), StructField(text,StringType,true), StructField(summary,StringType,true))
original_data: org.apache.spark.sql.DataFrame = [product: string, votes: int ... 4 more fields]


In [3]:
original_data.show(5)

+----------+-----+----+--------------------+--------------------+--------------------+
|   product|votes|rate|       original_text|                text|             summary|
+----------+-----+----+--------------------+--------------------+--------------------+
|0143065971|    4| 5.0|This is a masterp...|masterpiec someon...|Fantastic Book Ab...|
|0143065971|    1| 5.0|Great condition a...| great condit great |          Five Stars|
|1423600150|    1| 5.0|Excellent book on...|excel sauc fun tr...|           excellent|
|1423600150|    1| 5.0|          Great book|              great |          Five Stars|
|1423600150|    1| 5.0|   Great mexi stuff.|   great mexi stuff |          Five Stars|
+----------+-----+----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [17]:
original_data.select("summary","rate","votes").groupBy("summary").agg(mean("rate"),mean("votes"), count("summary")).orderBy(desc("count(summary)"))

+-----------+------------------+------------------+--------------+
|    summary|         avg(rate)|        avg(votes)|count(summary)|
+-----------+------------------+------------------+--------------+
| Five Stars| 4.999868593955322|1.0273981603153746|         15220|
| Four Stars| 4.000463714351959|1.0192441456063066|          4313|
|Three Stars| 3.001019367991845|1.0341488277268094|          1962|
|   One Star|1.0051380860629415|1.2190109184328837|          1557|
| Great book| 4.845056065239551|1.2252803261977574|           981|
|  Good read| 4.265864332603939|1.1816192560175054|           914|
|  Two Stars|2.0033632286995515|1.0661434977578474|           892|
| Great read| 4.753731343283582|1.2624378109452736|           804|
|      Great| 4.789115646258503|1.1319727891156464|           735|
|       Good| 4.152447552447552|1.1272727272727272|           715|
|    Awesome| 4.898697539797395|1.1331403762662808|           691|
|  Excellent|  4.92515923566879|1.5398089171974523|           

In [4]:
val binarizer = new Binarizer()
  .setInputCol("rate")
  .setOutputCol("label")
  .setThreshold(3.5);

// get n-grams
val tokenizer = new RegexTokenizer()
  .setInputCol("text")
  .setOutputCol("tokens")
  .setPattern("\\W");
val ngrams = new NGram()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("n-grams");

// calc tf-idf 
val tf = new HashingTF()
  .setInputCol(ngrams.getOutputCol)
  .setOutputCol("tf");
val idf = new IDF()
  .setInputCol(tf.getOutputCol)
  .setOutputCol("tf-idf")
  .setMinDocFreq(3);

// build the classifier
val classifierMod = new LogisticRegression()
  .setMaxIter(10)
  .setFeaturesCol(idf.getOutputCol)
  .setLabelCol(binarizer.getOutputCol);

// this is the pipeline that data follows to be evaluated
val pipeline = new Pipeline()
  .setStages(Array(binarizer, tokenizer, ngrams, tf, idf, classifierMod));

// a little of optimization: try different hyperparameters
val paramGrid = new ParamGridBuilder()
  .addGrid(classifierMod.regParam, Array(0.01, 0.05, 0.1))
  .addGrid(ngrams.n, Array(1, 2, 3))
  .build();

// do it with a cross validation on the train set (3 folds)
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3);

binarizer: org.apache.spark.ml.feature.Binarizer = Binarizer: uid=binarizer_1f2ecfd8bb5d
tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_4eca11e23721
ngrams: org.apache.spark.ml.feature.NGram = NGram: uid=ngram_08dd240d9a73, n=2
tf: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_1ed4df629fa6, binary=false, numFeatures=262144
idf: org.apache.spark.ml.feature.IDF = idf_6db681b5f222
classifierMod: org.apache.spark.ml.classification.LogisticRegression = logreg_9cdc54499216
pipeline: org.apache.spark.ml.Pipeline = pipeline_0234d6e5c92e
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	ngram_08dd240d9a73-n: 1,
	logreg_9cdc54499216-regParam: 0.01
}, {
	ngram_08dd240d9a73-n: 1,
	logreg_9cdc54499216-regParam: 0.05
}, {
	ngram_08dd240d9a73-n: 1,
	l...


In [13]:
spark.sql("SELECT summary, COUNT(*) FROM Original_data")

org.apache.spark.sql.AnalysisException:  grouping expressions sequence is empty, and 'original_data.`summary`' is not an aggregate function. Wrap '(count(1) AS `count(1)`)' in windowing function(s) or wrap 'original_data.`summary`' in first() (or first_value) if you don't care which value you get.;;

In [5]:
println("Training... ");
val model = cv.fit(original_data);
println("done!");

// print results
for (i <- 0 until model.avgMetrics.size) {
  println("\n\n");
  println(model.getEstimatorParamMaps(i));
  println(model.avgMetrics(i));
}


Training... 
done!



{
	ngram_08dd240d9a73-n: 1,
	logreg_9cdc54499216-regParam: 0.01
}
0.894217505803825



{
	ngram_08dd240d9a73-n: 1,
	logreg_9cdc54499216-regParam: 0.05
}
0.9034031006521369



{
	ngram_08dd240d9a73-n: 1,
	logreg_9cdc54499216-regParam: 0.1
}
0.905312185591742



{
	ngram_08dd240d9a73-n: 2,
	logreg_9cdc54499216-regParam: 0.01
}
0.8443155195102902



{
	ngram_08dd240d9a73-n: 2,
	logreg_9cdc54499216-regParam: 0.05
}
0.8644238243016412



{
	ngram_08dd240d9a73-n: 2,
	logreg_9cdc54499216-regParam: 0.1
}
0.872029019745533



{
	ngram_08dd240d9a73-n: 3,
	logreg_9cdc54499216-regParam: 0.01
}
0.6673848921888594



{
	ngram_08dd240d9a73-n: 3,
	logreg_9cdc54499216-regParam: 0.05
}
0.6837430757007262



{
	ngram_08dd240d9a73-n: 3,
	logreg_9cdc54499216-regParam: 0.1
}
0.6908928013842282


model: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_067743676650, bestModel=pipeline_0234d6e5c92e, numFolds=3
